In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /gdrive
/gdrive
Mounted at /content/drive


In [ ]:
class BoundingBox(object):
    """
    A 2D bounding box
    """

    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for point in points:
            # Set min coords
            if point['x'] < self.minx:
                self.minx = point['x']
            if point['y'] < self.miny:
                self.miny = point['y']
            # Set max coords
            if point['x'] > self.maxx:
                self.maxx = point['x']
            if point['y'] > self.maxy: #elif point['y'] > self.maxy: ('elif' caused faulty boxes) 
                self.maxy = point['y']

    @property
    def width(self):
        return self.maxx - self.minx

    @property
    def height(self):
        return self.maxy - self.miny

    def __repr__(self):
        return "BoundingBox({}, {}, {}, {})".format(
            self.minx, self.maxx, self.miny, self.maxy)

    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(other, BoundingBox):
            return self.minx == other.minx and self.miny == other.miny and self.maxy == other.maxy and self.maxx == other.maxx
        return False


In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt

def show_freehand_with_bbox(freehand_points, bbox):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim([0,700])
    ax.set_ylim([0,700])

    freehand_x = [float(i['x']) for i in freehand_points]
    freehand_y = [float(i['y']) for i in freehand_points]
    ax.plot(freehand_x, freehand_y, color="red", linewidth=2, label="freehand")

    bbox_x = [bbox.minx, bbox.maxx, bbox.maxx, bbox.minx, bbox.minx]
    bbox_y = [bbox.miny, bbox.miny, bbox.maxy, bbox.maxy, bbox.miny]
    ax.plot(bbox_x, bbox_y, color="blue", linewidth=2, label="bbox")

    print(f'Freehand points: {freehand_points}')
    print(f'Bounding box: {bbox}')

    plt.show()


def create_bounding_box(svg_object):
    if svg_object is not None:
        svg_array = json.loads(svg_object)
        b_boxes = []
        for svg in svg_array:
            points_array = np.array(svg['points'])  # svg_points_list convert to 2d array with numpy
            b_box = BoundingBox(points_array)
            b_boxes.append(b_box)
            #show_freehand_with_bbox(points_array, b_box)

        final_bbox = []
        unique_bbox = make_unique_bbox(b_boxes)
        for bbox in unique_bbox:
            temp_bbox = {
                'minx': bbox.minx,
                'miny': bbox.miny,
                'maxx': bbox.maxx,
                'maxy': bbox.maxy,
            }
            final_bbox.append(temp_bbox)

        return final_bbox


def make_unique_bbox(bounding_boxes):
    unique_bboxes = []
    for bbox in bounding_boxes:
        if bbox not in unique_bboxes:
            unique_bboxes.append(bbox)
    return unique_bboxes

In [10]:
import glob
import json
import os
import xml.etree.cElementTree as ET

# Change paths to train/validation accordingly
SOURCE_FILE_PATH = "/content/drive/My Drive/thyroid_nodule_detection/dataset/validation/" 
TARGET_JSON_FILE_PATH = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/validation.json"


# [ymin, xmin, ymax, xmax]
def xml_to_json(path):
    temp_cases = []  # dictionary case object list
    xml_files = glob.glob(path + '/*.xml', recursive=True)
    for xml_file in xml_files:
        temp = xml_file.split('/')
        file_name = temp[len(temp) - 1][:-4]
        tree = ET.parse(xml_file)
        root = tree.getroot()
        images = root.findall('mark')

        for image in images:
            if root.find('tirads').text is None:
              continue
            
            number = image.find('image').text
            image_id = file_name + '_' + number

            temp_bboxes = image.find('svg').text
            bounding_boxes = create_bounding_box(temp_bboxes)

            bboxes = []
            if bounding_boxes is not None:
                for bounding_box in bounding_boxes:
                    bbox = {
                        'x': bounding_box['minx'],
                        'y': bounding_box['miny'],
                        'w': bounding_box['maxx'] - bounding_box['minx'],
                        'h': bounding_box['maxy'] - bounding_box['miny'],
                    }
                    bboxes.append(bbox)
                case = {
                        'case_id': image_id,
                        'tirads': root.find('tirads').text,
                        'bboxes': bboxes
                }
                temp_cases.append(case)

    cases = {
        'cases': temp_cases
    }
    save_json(json.dumps(cases))


def save_json(final_json):
    text_file = open(TARGET_JSON_FILE_PATH, "w")
    text_file.write(final_json)
    text_file.close()


def main():
    xml_to_json(SOURCE_FILE_PATH)


main()